# Youtube CSV

In [1]:
api_key = "<API_KEY>"
channel_id = "<CHANNEL_ID"

In [3]:
from googleapiclient.discovery import build
youtube = build("youtube", "v3", developerKey=api_key)

In [4]:
def get_channel_videos(channel_id):
    res = youtube.channels().list(id=channel_id,
                                 part="contentDetails").execute()
    playlist_id = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                          part="snippet",
                                          maxResults=50,
                                          pageToken=next_page_token).execute()
        videos += res["items"]
        next_page_token = res.get("nextPageToken")
        
        if next_page_token is None:
            break
    
    return videos

In [5]:
videos = get_channel_videos(channel_id)

In [6]:
res = youtube.videos().list(id=videos[0]["snippet"]["resourceId"]["videoId"],
                           part="statistics").execute()

In [7]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

In [8]:
def get_videos_stats(video_ids):
    stats = []
    for i in range(0, len(video_ids), 50):
        res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   part='statistics').execute()
        stats += res['items']
    return stats

In [9]:
def get_videos_details(video_ids):
    details = []
    for i in range(0, len(video_ids), 50):
        res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   part='contentDetails').execute()
        details += res['items']
    return details

In [10]:
stats = get_videos_stats(video_ids)
details = get_videos_details(video_ids)

In [11]:
publishedAt = []
title = []
viewCount = []
likeCount = []
dislikeCount = []
commentCount = []
duration = []
description = []

for i in range(len(videos)):
    publishedAt.append(videos[i]['snippet']['publishedAt'])
    title.append(videos[i]['snippet']['title'])
    description.append(videos[i]['snippet']['description'])
    viewCount.append(stats[i]['statistics']['viewCount'])
    likeCount.append(stats[i]['statistics']['likeCount'])
    commentCount.append(stats[i]['statistics']['commentCount'])
    duration.append(details[i]['contentDetails']['duration'])

In [12]:
import pandas as pd
names = ['publishedAt', 'title', 'viewCount', 'likeCount', 'commentCount', 'duration', 'description']
df = pd.DataFrame(data=list(zip(publishedAt, title, viewCount, likeCount, commentCount, duration, description)),
                 columns=names)

In [13]:
df['viewCount'] = df['viewCount'].astype(int)
df['likeCount'] = df['likeCount'].astype(int)
df['commentCount'] = df['commentCount'].astype(int)
df.tail(1)

,publishedAt,title,viewCount,likeCount,commentCount,duration,description
3912,2018-10-04T10:06:05Z,빅토리의1번째주식단타매매영상,284,4,0,PT3M12S,현성바이탈


In [14]:
df.to_csv('youtube.csv', index=False)